# Key concepts behind SGD pagerank

# Understanding the Normalized Adjacency Matrix in SGD PageRank

In the presented computation, a distinct form of the normalized adjacency matrix has been employed. While this is not the traditional Laplacian, such normalized matrices have become increasingly prevalent, especially in the domain of graph-based learning methodologies, like graph convolutional networks (GCNs).

## 1. **Degrees Matrix**:

For each node in the graph, the sum of the weights of its edges is computed. This calculation is a generalization of the degree concept for weighted graphs. The formula for the degree of a node \(i\) in a weighted graph is given by:

$$ \text{degree}(i) = \sum_j \text{weight}(i, j) $$


Additionally, adding \(1\) to each node's degree represents the inclusion of a self-loop, ensuring every node retains a connection to itself.

## 2. **Inverse Square Root of Degrees**:

The inverse of the square root of the degrees is computed. This transformation is pivotal for normalization. The main objective behind this is to ensure nodes with a high degree don't exert a disproportionate influence. The formula to compute this for a node \(i\) is:

$$ \text{inv_sqrt_degree}(i) = \frac{1}{\sqrt{\text{degree}(i)}} $$


## 3. **Normalized Matrix**:

The final normalized matrix is created by multiplying the inverse square root diagonal matrix with the adjacency matrix, and then with the inverse square root matrix again:

$$ \text{normalized_matrix} = \text{inv_sqrt_deg_matrix} \times \text{neighborhood_matrix} \times \text{inv_sqrt_deg_matrix} $$

This process yields a symmetrically normalized adjacency matrix.

## **Implications of the Normalized Adjacency Matrix**:

- **Highlighting Local Structures**: The resulting PageRank scores derived using this matrix accentuate local structures more than a traditional approach. As such, nodes embedded within closely-knit communities or cliques might receive higher scores in this method as opposed to traditional PageRank.
  
- **Equitable Distribution**: The normalization mechanism ensures that nodes with an extremely high degree won't dominate the computation. This normalization ensures that there's a more balanced sharing of importance across nodes.

- **Connectivity Over Weight**: By using the normalized matrix, the emphasis is placed more on the patterns of connectivity rather than the raw edge weights. This means that the focus is more on the relational dynamics between nodes and their surrounding neighborhoods.

## **Understanding the Outcomes of SGD PageRank with the Normalized Adjacency Matrix**:

1. **Local and Global Importance**: The PageRank scores derived can be viewed as a balance between global significance and local neighborhood relevance. Nodes in tight communities might receive elevated importance due to the amplified local structure, while still maintaining a connection to the broader graph structure.

2. **Resilience to High-Degree Nodes**: Due to the symmetrical normalization, nodes with a high number of connections don't overshadow the computation. This mechanism ensures every node gets a fair consideration based on its connectivity patterns.

In summary, the integration of a normalized adjacency matrix in the SGD PageRank algorithm paves the way for a nuanced derivation of node importance. This approach takes into account both the global structure of the graph and the intricate dynamics of local neighborhoods.




## **Conclusions:**

1. **Using Normalized Laplacian:**
By using the normalized Laplacian in the context of single-cell data, you're essentially down-weighting the influence of cells that are highly connected (i.e., cells that are similar to many other cells). This can be beneficial if you believe that highly connected cells are not necessarily more "important" than less connected cells. By reducing their influence, you might highlight more unique or specialized cell states that could be overshadowed in a traditional PageRank computation.

2. **Interpretability and Biological Relevance:**
The key question is whether the rank scores, and the resulting ordering of cells, have biological relevance or offer insights into the cellular landscape. For example, do high-ranked cells correspond to critical transitional states in a developmental trajectory? Or do they highlight rare cell populations with distinct gene expression profiles?

# Stochastic Gradient Descent PageRank (SGD PageRank)

This document provides a detailed algorithmic description for a SGD PageRank implementation tailored for graphs derived from single-cell multiomic data. The aim is to prioritize the nodes (cells) based on their topological importance, refined by SGD to handle large-scale data efficiently.

## Preliminary Steps

### Dynamic Neighborhood Hopping

To capture the extended topological features of the graph, we implement dynamic neighborhood hopping:

$$
A^{(\alpha)} = A^{\alpha}
$$

where \( A \) is the adjacency matrix, and \( \alpha \) is the predefined number of hops ensuring all nodes have direct paths to all other nodes within \( \alpha \) hops.

### Scaling Factor Calculation (Si)

The scaling factor for each node is calculated to down-weight the influence of highly connected nodes:

$$
S_i = \frac{1}{\text{degree}(i)} + C(D_i)
$$

where \( C(D_i) \) represents a correction based on the node's adjacency set.

### Matrix Scaling (Mij)

We scale the k-nearest neighbors (KNN) matrix by applying a dot product of \( S_i \) to both incoming and outgoing connections:

$$
M_{ij} = S_i \cdot M \cdot S_i
$$

where \( M \) is the KNN matrix.

## SGD PageRank Algorithm

The main algorithm proceeds as follows:

```python
def SGDpagerank(M, num_iterations, mini_batch_size, initial_learning_rate, tolerance, d, full_batch_update_iters, dip_window, plateau_iterations, sampling_method, init_vect=None, **kwargs):
    # ... (implementation details)
```

### Learning Rate (Alpha)

The learning rate is updated at each iteration to ensure convergence:

$$
\alpha = \frac{1}{1 + \text{decay_rate} \cdot \text{iteration}}
$$

### PageRank Initialization

A random rank vector \( v \) is initialized and normalized:

$$
v = \frac{\text{rand}(N, 1)}{\| \text{rand}(N, 1) \|_1}
$$

### Mini-Batch SGD Iterations

At each iteration, a subset of nodes is selected, and the PageRank vector is updated:

$$
v_{\text{mini_batch}} = d \cdot (\alpha \cdot M_{\text{mini_batch}} @ v) + \left(\frac{1 - d}{N}\right)
$$

where \( @ \) denotes matrix-vector multiplication.

## Convergence Check

We monitor the L2 norm of the PageRank vector difference for convergence:

$$
\| v_{\text{iter}} - v_{\text{prev}} \|_2 < \text{tolerance}
$$

## Full-Batch Updates

After the main SGD iterations, we perform a number of full-batch updates for fine-tuning:

$$
v = d \cdot (M @ v) + \left(\frac{1 - d}{N}\right)
$$

## Post-Processing

### Softmax Transformation

Once the PageRank vector is obtained, a softmax transformation is applied to obtain a probability distribution for sampling:

$$
P_i = \frac{e^{v_i}}{\sum_{j=1}^N e^{v_j}}
$$

### Sampling Strategy

Finally, we perform sampling from the softmax-transformed PageRank scores to select nodes:

- Sampling is done 100 times using the PageRank scores.
- The observed probability of sampling is used to derive the output node indices at a given proportion.

This concludes the detailed algorithmic description of the SGD PageRank methodology.
